In [2]:
from datetime import date

from selenium import webdriver
from selenium.webdriver import ChromeOptions
from bs4 import BeautifulSoup
from sys import stderr
from datetime import timedelta
import time
from ipywidgets import widgets
from IPython.display import display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
class Projekcija:
    def __init__(self, filmm, vreme, bioskop, cena):
        self._link = None
        self._film = filmm
        self._vreme = vreme
        self._cena = cena
        self._bioskop = bioskop

    def __str__(self):
        return f'Film: {self._film}'

    @property
    def link(self):
        return self._link

    @link.setter
    def link(self, value):
        self._link = value
        
    @property
    def film(self):
        return self._film

    @film.setter
    def film(self, value):
        self._film = value

    @property
    def vreme(self):
        return self._vreme

    @vreme.setter
    def vreme(self, value):
        self._vreme = value

    @property
    def bioskop(self):
        return self._bioskop

    @bioskop.setter
    def bioskop(self, value):
        self._bioskop = value

    @property
    def cena(self):
        return self._cena

    @cena.setter
    def cena(self, value):
        self._cena = value


In [4]:
def get_chrome_web_driver():
    """
    Creates and returns a Selenium web driver for Chrome web-browser

    :return: Selenium web driver for Chrome browser
    """
    options = ChromeOptions()
    options.add_argument('headless') # to prevent browser from opening when extracting content of a web page
    return webdriver.Chrome(options=options)

In [5]:
def citanje_stranice(webdriver, url):
    webdriver.get(url)
    page_content = webdriver.page_source
    page_soup = BeautifulSoup(page_content, features='html.parser')
    if not page_soup:
        raise RuntimeError(f"Could not parse content from URL: {url}. Cannot proceed!")
    supa = page_soup.find('div', class_='movie-list')
    try:
        supa.find('ul').find_all('li')
    except Exception as err:
        return None
    return supa

In [6]:
def citanje_stranice2(webdriver, url):
    webdriver.get(url)
    page_content = webdriver.page_source
    page_soup = BeautifulSoup(page_content, features='html.parser')
    if not page_soup:
        raise RuntimeError(f"Could not parse content from URL: {url}. Cannot proceed!")
    #try:
   #     supa.find('ul').find_all('li')
   # except Exception as err:
   #     return None
    return page_soup

In [7]:
def prva(url):
    listaFilmova=list()
    webdriver = get_chrome_web_driver()
    if not webdriver:
        raise RuntimeError("An error occurred while setting up web driver!")
    supa=None
    uspeh=False
    n=0
    while not uspeh:
        supa=citanje_stranice(webdriver, url)
        if not supa is None:
            uspeh=True
            break
        n=n+1
        if n==4:
            raise Exception()
    for li in supa.find('ul').find_all('li'):
        figcaption = li.find('figcaption', class_='l-entity__figure-caption')
        if figcaption:
            text = figcaption.get_text()
            p=Projekcija(text, None, None, None)
            listaFilmova.append(p)
        else:
            print("Element nije pronađen!")
    return listaFilmova
    

In [8]:
def druga(url):
    listaFilmova=list()
    webdriver = get_chrome_web_driver()
    if not webdriver:
        raise RuntimeError("An error occurred while setting up web driver!")
    supa=None
    uspeh=False
    n=0
    while not uspeh:
        supa=citanje_stranice2(webdriver, url)
        if not supa is None:
            uspeh=True
            break
        n=n+1
        if n==4:
            raise Exception()
    for bioskop in supa.findAll('div', class_='sc-cSHVUG sc-jzgbtB b-entity-content gRedAN'):
        for proj in bioskop.find('ul').find_all('li'):
            if proj:
                b= bioskop.find('a', class_='b-entity-content__title b-entity-content__link')
                b=b.text
                proj=proj.find('a', class_='sc-kEYyzF l-tickets__item-link khlFkZ')
                link1=proj.get('href')
                link=f'https://www.cineplexx.rs{link1}/tickets'
                status = proj.get('status')
                vreme=proj.find('p', class_='l-tickets__item-time').get_text()
                projekcija=Projekcija(odabrani_film, vreme, b, None)
                projekcija.link=link
                if status=="green" or status=="yellow":
                    listaFilmova.append(projekcija)
                #text = figcaption.get_text()
                #p=Projekcija(text, None, None, None)
                
            else:
                print("Element nije pronađen!")
    return listaFilmova

In [9]:
url="https://www.cineplexx.rs/film?category=now&location=0&date=2025-01-25"
listaFilmova=None
try:
    print("Lista filmova:")
    listaFilmova= prva(url)
    broj=1
    for film in listaFilmova:
        print(str(broj)+". "+film.film)
        broj=broj+1
except RuntimeError as err:
    stderr.write(f"Terminating the program due to the following runtime error:\n{err}")
except Exception as err:
    print("Greska")
    stderr.write(f"Terminating the program due to the following runtime error:\n{err}")

Lista filmova:
1. Bob Dylan: Potpuni neznanac
2. Bratstvo lopova 2: Panter
3. Čovek vuk
4. Deda Mraz: Superheroj
5. Gladijator II
6. Istinski bol
7. Izolacija
8. Jeretik
9. Mali Alan
10. Meda Pedington u džungli
11. Mudbrick: Kult mrtvih
12. Mufasa: Kralj lavova
13. Nosferatu
14. Pored nas
15. Požuda
16. Rizičan let
17. Sonikov film 3
18. Susedna soba
19. Vajana 2
20. Volja sinovljeva
21. Vreme koje imamo
22. Žena u praznoj sobi
23. Zlica


In [48]:
try:
    broj = input("Unesite broj filma koji zelite da izaberete: ")
    print("Uneli ste:", str(broj))
    print("Odabrali ste film: "+listaFilmova[int(broj)-1].film)
    odabrani_film=listaFilmova[int(broj)-1]
except Exception as err:
    print("Pogresan unos")

Uneli ste: 2
Odabrali ste film: Bratstvo lopova 2: Panter


In [51]:
try:
    dan = input("Ako zelite informacije za danas unesite 0, za sutra 1, za dva dana 2, do 4: ")
    print("Uneli ste:", str(dan))
    dan=int(dan)
    if dan<0 or dan>4:
        print("Pogresan unos")
    else:
        if dan==0:
            print("Izabrali ste danas")
        elif dan==1:
            print("Izabrali ste sutra")
        elif dan==2:
            print("Izabrali ste prekosutra")
        elif dan==3:
            print("Izabrali ste za 3 dana")
        elif dan==1:
            print("Izabrali ste za 4 dana")
except Exception as err:
    print("Pogresan unos")

Uneli ste: 0
Izabrali ste danas


In [12]:


f1=odabrani_film.film.split(" ")
ime=""
for rec in f1:
    rec=rec.strip(":")
    ime=ime+"-"+rec
ime=ime.strip("-")
datum=date.today()
datum=datum + timedelta(days=dan)
datum=datum.strftime("%Y-%m-%d")  

'2025-01-26'

In [13]:
url_filma=f"https://www.cineplexx.rs/film/{ime}?date={datum}&location=0"

'https://www.cineplexx.rs/film/Bratstvo-lopova-2-Panter?date=2025-01-26&location=0'

In [38]:
listaPotencijalnih=None
try:
    print("Lista projekcija izabranog filma, izabranog dana:")
    uspeh=False
    t=0
    while not uspeh:
        listaPotencijalnih= druga(url_filma)
        if len(listaPotencijalnih)>0:
            uspeh=True
        t=t+1
        if t>4:
            raise Exception("GRESKAA")
    #ako je duzina liste 0 onda ponovi za error, mozda
    broj=1
    for film1 in listaPotencijalnih:
        print(str(broj)+". "+odabrani_film.film+" "+film1.bioskop+" "+film1.vreme)
        print(film1.link)
        broj=broj+1
except RuntimeError as err:
    stderr.write(f"Terminating the program due to the following runtime error:\n{err}")
except Exception as err:
    print("Greska")
    stderr.write(f"Terminating the program due to the following runtime error:\n{err}")

Lista projekcija izabranog filma, izabranog dana:
1. Bratstvo lopova 2: Panter CINEPLEXX 4D DELTA CITY 20:20
https://www.cineplexx.rs/purchase/wizard/1111-56354/tickets
2. Bratstvo lopova 2: Panter CINEPLEXX UŠĆE SHOPPING CENTER 18:40
https://www.cineplexx.rs/purchase/wizard/1113-95789/tickets
3. Bratstvo lopova 2: Panter CINEPLEXX UŠĆE SHOPPING CENTER 21:10
https://www.cineplexx.rs/purchase/wizard/1113-95737/tickets
4. Bratstvo lopova 2: Panter Cineplexx BIG Beograd 20:45
https://www.cineplexx.rs/purchase/wizard/1115-68734/tickets
5. Bratstvo lopova 2: Panter CINEPLEXX BEO SHOPPING CENTER 21:00
https://www.cineplexx.rs/purchase/wizard/1118-48404/tickets
6. Bratstvo lopova 2: Panter CINEPLEXX GALERIJA BEOGRAD 20:00
https://www.cineplexx.rs/purchase/wizard/1119-43140/tickets
7. Bratstvo lopova 2: Panter CINEPLEXX GALERIJA BEOGRAD 22:30
https://www.cineplexx.rs/purchase/wizard/1119-43160/tickets


In [46]:
try:
    #driver = webdriver.Chrome()  #da se vidi chrome otvoren
    driver = get_chrome_web_driver()
    for film1 in listaPotencijalnih:
        driver.get(film1.link)
        #popup    nekad nema, nekad ima
        try:
            button = driver.find_element(By.ID, "CybotCookiebotDialogBodyButtonDecline")
            button.click()
            driver.implicitly_wait(2)
        except Exception as err:
            #print("Nema za kuki")
            driver.implicitly_wait(2)
        
        #jedna osoba
        button = driver.find_elements(By.CLASS_NAME, "sc-feJyhm.kDucxu")[1]
        button.click()
        driver.implicitly_wait(2)
        #dugme sledece
        button = driver.find_elements(By.CSS_SELECTOR, ".sc-bdVaJa.LRCVX")[1]
        driver.execute_script("arguments[0].click();", button)
        driver.implicitly_wait(5)
    
        #odabir mesta    nova stranica
        buttons = driver.find_elements(By.CSS_SELECTOR, ".sc-cpmKsF.l-seats__item[tabindex]")
        nadjeno_mesto=False
        for button in buttons:
            try:
                #proveri da li je vip
                img_element = button.find_element(By.TAG_NAME, "img")
                src = img_element.get_attribute("src")
                if src!="https://app.cineplexx.rs/static/area_categories/free.svg":
                    continue
                ##https://app.cineplexx.rs/static/area_categories/vip.svg
                button.click()
                driver.implicitly_wait(2)
                strong_element = driver.find_element(By.CSS_SELECTOR, "div.sc-kAzzGY.sc-ipZHIp.bYIrLb strong")
                cena1 = strong_element.text
                nadjeno_mesto=True
                cena1=cena1.split(",")
                cena1=cena1[0]
                cena1=int(cena1)
                if cena1==0:
                    continue
                film1.cena=cena1
                break
            except Exception as err:
                print()#ne treba nista da stamp
    listaPotencijalnih.sort(key=lambda x: x.cena, reverse=True)
    for film1 in listaPotencijalnih:
        print(odabrani_film.film+" cena "+str(film1.cena)+" "+film1.vreme+" "+ film1.bioskop+" "+film1.link)
    razlika=listaPotencijalnih[0].cena-listaPotencijalnih[-1].cena
    print("Danas mozete da ustedite najvise "+str(razlika)+" dinara")
except Exception as err:
    print("Greska")
    stderr.write(f"Terminating the program due to the following runtime error:\n{err}")












Bratstvo lopova 2: Panter cena 880 20:00 CINEPLEXX GALERIJA BEOGRAD https://www.cineplexx.rs/purchase/wizard/1119-43140/tickets
Bratstvo lopova 2: Panter cena 880 22:30 CINEPLEXX GALERIJA BEOGRAD https://www.cineplexx.rs/purchase/wizard/1119-43160/tickets
Bratstvo lopova 2: Panter cena 860 18:40 CINEPLEXX UŠĆE SHOPPING CENTER https://www.cineplexx.rs/purchase/wizard/1113-95789/tickets
Bratstvo lopova 2: Panter cena 860 21:10 CINEPLEXX UŠĆE SHOPPING CENTER https://www.cineplexx.rs/purchase/wizard/1113-95737/tickets
Bratstvo lopova 2: Panter cena 830 20:20 CINEPLEXX 4D DELTA CITY https://www.cineplexx.rs/purchase/wizard/1111-56354/tickets
Bratstvo lopova 2: Panter cena 830 20:45 Cineplexx BIG Beograd https://www.cineplexx.rs/purchase/wizard/1115-68734/tickets
Bratstvo lopova 2: Panter cena 830 21:00 CINEPLEXX BEO SHOPPING CENTER https://www.cineplexx.rs/purchase/wizard/1118-48404/tickets
Danas mozete da ustedite najvise 50 dinara
